In [1]:
import sys, os, csv
import math
import pandas as pd
import numpy as np
import nltk
import time
import re
import json
import requests
import datetime

In [2]:
gop = requests.get('http://elections.huffingtonpost.com/pollster/api/charts/2016-national-gop-primary.json')

In [3]:
gop = gop.json()

In [4]:
gop_poll = gop['estimates_by_date']

Merge and aggregate all values for Polling Results
Note: Looks like only loaded on days with new poll update

In [5]:
# Works, Daily as Rows
# pollsDaily = pd.DataFrame()
# for i in range(0,3):
for i in range(0,len(gop_poll)):
    # print(gop_poll[i]['date'])
    date = gop_poll[i]['date']
    DF = gop_poll[i]['estimates']
    DF = pd.DataFrame(DF)
    # DF.columns = ['choice',  date]
    DF.set_index('choice',inplace=True)
    DF = DF.T
    DF['date'] = gop_poll[i]['date']
    # del DF['']
    DF.index = [i]
    if i == 0:
        pollsDaily = DF
        #print(DF)
    else:
        #print(DF)
        pollsDaily = pollsDaily.append(DF)
# pollsDaily

In [6]:
demChart = '2016-national-democratic-primary'

In [7]:
dem = requests.get('http://elections.huffingtonpost.com/pollster/api/charts/'+demChart+'.json')

In [8]:
dem = dem.json()

In [9]:
dem_poll = dem['estimates_by_date']

In [10]:
# Works, Daily as Rows
# pollsDaily = pd.DataFrame()
# for i in range(0,3):
for i in range(0,len(dem_poll)):
    # print(dem_poll[i]['date'])
    date = dem_poll[i]['date']
    DF = dem_poll[i]['estimates']
    DF = pd.DataFrame(DF)
    # DF.columns = ['choice',  date]
    DF.set_index('choice',inplace=True)
    DF = DF.T
    DF['date'] = dem_poll[i]['date']
    # del DF['']
    DF.index = [i]
    if i == 0:
        pollsDaily_dem = DF
        #print(DF)
    else:
        #print(DF)
        pollsDaily_dem = pollsDaily_dem.append(DF)
# pollsDaily

In [11]:
# Modify Other and Undecided Labels so as to clarify from Dem or GOP list
pollsDaily.columns
# Modify Other and Undecided Labels so as to clarify from Dem or GOP list
pollsDaily = pollsDaily.rename(columns = {'Undecided':'Undecided_GOP'})
pollsDaily = pollsDaily.rename(columns = {'Refused':'Refused_GOP'})
pollsDaily = pollsDaily.rename(columns = {'Other':'Other_GOP'})

pollsDaily.columns

Index(['Bolton', 'Bush', 'Carson', 'Christie', 'Cruz', 'Fiorina', 'Gilmore',
       'Graham', 'Haley', 'Huckabee', 'Jindal', 'Kasich', 'Martinez',
       'McDonnell', 'Other_GOP', 'Palin', 'Pataki', 'Pence', 'Perry',
       'Portman', 'Rand Paul', 'Refused_GOP', 'Rice', 'Romney', 'Rubio',
       'Ryan', 'Santorum', 'Trump', 'Undecided_GOP', 'Walker', 'date'],
      dtype='object')

In [12]:
# Modify Other and Undecided Labels so as to clarify from Dem or GOP list
pollsDaily_dem.columns
# Modify Other and Undecided Labels so as to clarify from Dem or GOP list
pollsDaily_dem = pollsDaily_dem.rename(columns = {'Undecided':'Undecided_Dem'})
pollsDaily_dem = pollsDaily_dem.rename(columns = {'Refused':'Refused_Dem'})
pollsDaily_dem = pollsDaily_dem.rename(columns = {'Other':'Other_Dem'})

pollsDaily_dem.columns

Index(['Biden', 'Booker', 'Chafee', 'Clinton', 'Cuomo', 'Dean', 'Gillibrand',
       'Kerry', 'Lessig', 'O'Malley', 'Other_Dem', 'Patrick', 'Sanders',
       'Schweitzer', 'Undecided_Dem', 'Villaraigosa', 'Warner', 'Warren',
       'Webb', 'date'],
      dtype='object')

In [13]:
master = pd.merge(pollsDaily, pollsDaily_dem, on= 'date')

In [14]:
master.columns

Index(['Bolton', 'Bush', 'Carson', 'Christie', 'Cruz', 'Fiorina', 'Gilmore',
       'Graham', 'Haley', 'Huckabee', 'Jindal', 'Kasich', 'Martinez',
       'McDonnell', 'Other_GOP', 'Palin', 'Pataki', 'Pence', 'Perry',
       'Portman', 'Rand Paul', 'Refused_GOP', 'Rice', 'Romney', 'Rubio',
       'Ryan', 'Santorum', 'Trump', 'Undecided_GOP', 'Walker', 'date', 'Biden',
       'Booker', 'Chafee', 'Clinton', 'Cuomo', 'Dean', 'Gillibrand', 'Kerry',
       'Lessig', 'O'Malley', 'Other_Dem', 'Patrick', 'Sanders', 'Schweitzer',
       'Undecided_Dem', 'Villaraigosa', 'Warner', 'Warren', 'Webb'],
      dtype='object')

In [15]:
master.set_index('date', inplace = True)

In [16]:
master

,Bolton,Bush,Carson,Christie,Cruz,Fiorina,Gilmore,Graham,Haley,Huckabee,...,O'Malley,Other_Dem,Patrick,Sanders,Schweitzer,Undecided_Dem,Villaraigosa,Warner,Warren,Webb
date,,,,,,,,,,,,,,,,,,,,,
2016-05-08,NaN,NaN,NaN,NaN,23.41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.68,NaN,4.83,NaN,NaN,NaN,NaN
2016-05-04,NaN,NaN,NaN,NaN,24.39,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.60,NaN,4.91,NaN,NaN,NaN,NaN
2016-05-02,NaN,NaN,NaN,NaN,24.85,NaN,NaN,NaN,NaN,NaN,...,NaN,3.99,NaN,41.67,NaN,4.91,NaN,NaN,NaN,NaN
2016-05-01,NaN,NaN,NaN,NaN,25.08,NaN,NaN,NaN,NaN,NaN,...,NaN,3.98,NaN,41.70,NaN,4.92,NaN,NaN,NaN,NaN
2016-04-29,NaN,NaN,NaN,NaN,25.52,NaN,NaN,NaN,NaN,NaN,...,NaN,3.96,NaN,41.92,NaN,4.87,NaN,NaN,NaN,NaN
2016-04-28,NaN,NaN,NaN,NaN,25.74,NaN,NaN,NaN,NaN,NaN,...,NaN,3.95,NaN,42.05,NaN,4.85,NaN,NaN,NaN,NaN
2016-04-27,NaN,NaN,NaN,NaN,25.96,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,42.20,NaN,4.84,NaN,NaN,NaN,NaN
2016-04-26,NaN,NaN,NaN,NaN,26.18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,42.36,NaN,4.84,NaN,NaN,NaN,NaN
2016-04-24,NaN,NaN,NaN,NaN,26.62,NaN,NaN,NaN,NaN,NaN,...,NaN,3.94,NaN,42.64,NaN,4.88,NaN,NaN,NaN,NaN


In [27]:
master.to_csv('HuffPost_Aggregated_National_Poll.csv')

### Adding linear interpolation for missing date values between polls

In [24]:
master = pd.read_csv('HuffPost_Aggregated_National_Poll.csv')

In [25]:
maxDate = master.date.max()
minDate = master.date.min()

In [26]:
minDate = datetime.datetime.strptime(minDate, '%Y-%m-%d')
maxDate = datetime.datetime.strptime(maxDate, '%Y-%m-%d')

In [27]:
numdays = maxDate-minDate
date_list = [maxDate - datetime.timedelta(days=x) for x in range(0, numdays.days)]

In [28]:
len(date_list)

1253

In [62]:
samp = master[['Clinton','Trump', 'date']]

In [63]:
samp

,Clinton,Trump,date
0,51.63,56.40,2016-05-08
1,51.42,54.16,2016-05-04
2,51.26,53.09,2016-05-02
3,51.21,52.55,2016-05-01
4,51.02,51.51,2016-04-29
5,50.91,50.99,2016-04-28
6,50.79,50.47,2016-04-27
7,50.66,49.96,2016-04-26
8,50.41,48.94,2016-04-24
9,50.14,47.96,2016-04-22


In [71]:
date_list[0].strftime('%m/%d/%y')

'05/08/16'

In [74]:
type(samp.loc[0,'date'])

str

In [34]:
date_listDF = pd.DataFrame(date_list, columns = ['date_dt'])

In [35]:
date_listDF[0:10]

,date_dt
0,2016-05-08
1,2016-05-07
2,2016-05-06
3,2016-05-05
4,2016-05-04
5,2016-05-03
6,2016-05-02
7,2016-05-01
8,2016-04-30
9,2016-04-29


In [44]:
pd.merge(date_listDF, master, how='left', on='date_dt')

,date_dt,date,Bolton,Bush,Carson,Christie,Cruz,Fiorina,Gilmore,Graham,...,O'Malley,Other_Dem,Patrick,Sanders,Schweitzer,Undecided_Dem,Villaraigosa,Warner,Warren,Webb
0,2016-05-08,2016-05-08,NaN,NaN,NaN,NaN,23.41,NaN,NaN,NaN,...,NaN,NaN,NaN,41.68,NaN,4.83,NaN,NaN,NaN,NaN
1,2016-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-05-04,2016-05-04,NaN,NaN,NaN,NaN,24.39,NaN,NaN,NaN,...,NaN,NaN,NaN,41.60,NaN,4.91,NaN,NaN,NaN,NaN
5,2016-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016-05-02,2016-05-02,NaN,NaN,NaN,NaN,24.85,NaN,NaN,NaN,...,NaN,3.99,NaN,41.67,NaN,4.91,NaN,NaN,NaN,NaN
7,2016-05-01,2016-05-01,NaN,NaN,NaN,NaN,25.08,NaN,NaN,NaN,...,NaN,3.98,NaN,41.70,NaN,4.92,NaN,NaN,NaN,NaN
8,2016-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016-04-29,2016-04-29,NaN,NaN,NaN,NaN,25.52,NaN,NaN,NaN,...,NaN,3.96,NaN,41.92,NaN,4.87,NaN,NaN,NaN,NaN


In [42]:
date_dt = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in master.date]

In [43]:
master['date_dt'] = np.array(date_dt)

In [22]:
master.loc[0,'date_dt'] - master.loc[1,'date_dt']

Timedelta('4 days 00:00:00')

In [23]:
len(master)

231

In [88]:
del samp['date_new']

In [90]:
master['date_new'] = np.array(samp_new_date)

In [91]:
master

,date,Bolton,Bush,Carson,Christie,Cruz,Fiorina,Gilmore,Graham,Haley,...,Other_Dem,Patrick,Sanders,Schweitzer,Undecided_Dem,Villaraigosa,Warner,Warren,Webb,date_new
0,2016-05-08,NaN,NaN,NaN,NaN,23.41,NaN,NaN,NaN,NaN,...,NaN,NaN,41.68,NaN,4.83,NaN,NaN,NaN,NaN,2016-05-08
1,2016-05-04,NaN,NaN,NaN,NaN,24.39,NaN,NaN,NaN,NaN,...,NaN,NaN,41.60,NaN,4.91,NaN,NaN,NaN,NaN,2016-05-04
2,2016-05-02,NaN,NaN,NaN,NaN,24.85,NaN,NaN,NaN,NaN,...,3.99,NaN,41.67,NaN,4.91,NaN,NaN,NaN,NaN,2016-05-02
3,2016-05-01,NaN,NaN,NaN,NaN,25.08,NaN,NaN,NaN,NaN,...,3.98,NaN,41.70,NaN,4.92,NaN,NaN,NaN,NaN,2016-05-01
4,2016-04-29,NaN,NaN,NaN,NaN,25.52,NaN,NaN,NaN,NaN,...,3.96,NaN,41.92,NaN,4.87,NaN,NaN,NaN,NaN,2016-04-29
5,2016-04-28,NaN,NaN,NaN,NaN,25.74,NaN,NaN,NaN,NaN,...,3.95,NaN,42.05,NaN,4.85,NaN,NaN,NaN,NaN,2016-04-28
6,2016-04-27,NaN,NaN,NaN,NaN,25.96,NaN,NaN,NaN,NaN,...,NaN,NaN,42.20,NaN,4.84,NaN,NaN,NaN,NaN,2016-04-27
7,2016-04-26,NaN,NaN,NaN,NaN,26.18,NaN,NaN,NaN,NaN,...,NaN,NaN,42.36,NaN,4.84,NaN,NaN,NaN,NaN,2016-04-26
8,2016-04-24,NaN,NaN,NaN,NaN,26.62,NaN,NaN,NaN,NaN,...,3.94,NaN,42.64,NaN,4.88,NaN,NaN,NaN,NaN,2016-04-24
9,2016-04-22,NaN,NaN,NaN,NaN,27.04,NaN,NaN,NaN,NaN,...,3.90,NaN,42.94,NaN,4.89,NaN,NaN,NaN,NaN,2016-04-22


In [93]:
master.loc[0,'date_new']

Timestamp('2016-05-08 00:00:00')

In [94]:
samp[0:2]

,Clinton,Trump,date,date_new
0,51.63,56.40,2016-05-08,2016-05-08
1,51.42,54.16,2016-05-04,2016-05-04


In [97]:
td = samp.loc[1,'date_new'] - samp.loc[0,'date_new'] 

In [98]:
td.days

-4

In [99]:
delta = samp.loc[1,'Clinton'] - samp.loc[0,'Clinton'] 

In [100]:
delta

-0.21000000000000085

In [102]:
day_delta = delta/4

In [106]:
base = samp.loc[0,'date_new']

In [119]:
basePoll = samp.loc[0,'Clinton']

In [120]:
[(base + datetime.timedelta(i), basePoll + i*day_delta) for i in range(0,abs(td.days))]

[(Timestamp('2016-05-08 00:00:00'), 51.630000000000003),
 (Timestamp('2016-05-09 00:00:00'), 51.577500000000001),
 (Timestamp('2016-05-10 00:00:00'), 51.525000000000006),
 (Timestamp('2016-05-11 00:00:00'), 51.472500000000004)]

In [113]:
abs(td.days)

4

In [114]:
list(range(0,abs(td.days)))

[0, 1, 2, 3]

In [122]:
date_list

[datetime.datetime(2016, 5, 8, 0, 0),
 datetime.datetime(2016, 5, 7, 0, 0),
 datetime.datetime(2016, 5, 6, 0, 0),
 datetime.datetime(2016, 5, 5, 0, 0),
 datetime.datetime(2016, 5, 4, 0, 0),
 datetime.datetime(2016, 5, 3, 0, 0),
 datetime.datetime(2016, 5, 2, 0, 0),
 datetime.datetime(2016, 5, 1, 0, 0),
 datetime.datetime(2016, 4, 30, 0, 0),
 datetime.datetime(2016, 4, 29, 0, 0),
 datetime.datetime(2016, 4, 28, 0, 0),
 datetime.datetime(2016, 4, 27, 0, 0),
 datetime.datetime(2016, 4, 26, 0, 0),
 datetime.datetime(2016, 4, 25, 0, 0),
 datetime.datetime(2016, 4, 24, 0, 0),
 datetime.datetime(2016, 4, 23, 0, 0),
 datetime.datetime(2016, 4, 22, 0, 0),
 datetime.datetime(2016, 4, 21, 0, 0),
 datetime.datetime(2016, 4, 20, 0, 0),
 datetime.datetime(2016, 4, 19, 0, 0),
 datetime.datetime(2016, 4, 18, 0, 0),
 datetime.datetime(2016, 4, 17, 0, 0),
 datetime.datetime(2016, 4, 16, 0, 0),
 datetime.datetime(2016, 4, 15, 0, 0),
 datetime.datetime(2016, 4, 14, 0, 0),
 datetime.datetime(2016, 4, 13, 0